<a href="https://colab.research.google.com/github/Seyviour/fraud-mail-bert/blob/master/fraud_mail_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir data
!wget https://raw.githubusercontent.com/Seyviour/fraud-mail-bert/master/fraud_email_.csv >> '/content/data'

/bin/bash: /content/data: Is a directory


In [ ]:
!ls data

In [ ]:
!cd data && wget -q https://raw.githubusercontent.com/Seyviour/fraud-mail-bert/master/fraud_email_.csv
!ls data

fraud_email_.csv


In [ ]:
import os
import pandas as pd

In [ ]:
master_frame = pd.read_csv('/content/data/fraud_email_.csv')
master_frame.head(5)




,Text,Class
0,Supply Quality China's EXCLUSIVE dimensions at...,1
1,over. SidLet me know. Thx.,0
2,"Dear Friend,Greetings to you.I wish to accost ...",1
3,MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....,1
4,Not a surprising assessment from Embassy.,0


In [ ]:
# cast text column to string and class column to int

master_frame['Text'] = master_frame['Text'].astype(str)
master_frame['Class'] = master_frame['Class'].astype(int)

# Lower-case text-portion for BERT

master_frame['Text'].str.lower()

NameError: ignored

In [ ]:
train_df = master_frame.sample(frac=0.7, random_state=42)
test_df = master_frame.loc[~master_frame.index.isin(train_df.index)]

train_df.reset_index(drop = True, inplace=True)
test_df.reset_index(drop = True, inplace=True)

In [ ]:
# Ensure that the data in the columns is of the correct type (string for 'Text', and int for 'Class')

train_df['Text'] = train_df['Text'].astype(str)
train_df['Class'] = train_df['Class'].astype(int)
test_df['Text'] = test_df['Text'].astype(str)
test_df['Class'] =test_df['Class'].astype(int)


###lowercase the text column


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
test_df.head(10)

,Text,Class
0,over. SidLet me know. Thx.,0
1,Not a surprising assessment from Embassy.,0
2,Monica -Huma Abedin <Huma@clintonemail.com>Tue...,0
3,Pis print.H <hrod17@clintonemail.com>Thursday ...,0
4,Dear Tom--H <hrod17@clintonemail.com>Friday De...,0
5,FYI. Thanks again for signing the book ---- an...,0
6,Not sure about SRAP. I'll forward it on. Hope ...,0
7,"Dear Sir/Madam,Compliments of the season. It ...",1
8,The letter is good to go w me. HRC.,0
9,"DEAR SIR/MADAM,MY NAME IS ENGINEER. TONY EDEM ...",1


In [ ]:
%%capture
!pip install simpletransformers

The apex installation below takes a wee while to complete. Please be patient with it :)
simpletransformers text classification won't work without it and it's the fastest I've seen BERT run (granted that isn't a lot of times) for what it's worth.

In [ ]:
%%capture
! git clone https://github.com/NVIDIA/apex
! cd apex
! cd apex &&  pip3 install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

We'll be using the classification model from simpletransformers. Importing it directly here for cleaner access


In [ ]:
from simpletransformers.classification import ClassificationModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Classification using BERT, specifically 'bert-base-uncased' 

In [ ]:
model = ClassificationModel('bert', 'bert-base-uncased', 2, use_cuda = True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Finally we get to train the model (on the train_df) we separated earlier. Thank you simpletransformers team for making this easy. I'm cry :]

In [ ]:
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:282: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc = sklearn.metrics.classification_report)

NameError: ignored

In [ ]:
result['acc']

In [ ]:
result

In [ ]:
predictions, raw_outputs = model.predict(['i am a nigerian prince with tonnes of money to give you for free just send'])

In [ ]:
predictions

In [ ]:
import sklearn